In [1]:
import fitz  # PyMuPDF
import json

pdf_path = "AFTERBURNER CONTROL_Chapter 3.pdf"
doc = fitz.open(pdf_path)

page_data = []

for i in range(len(doc)):
    text = doc[i].get_text()
    page_data.append({
        "page": i + 1,
        "text": text
    })

# save as JSON
with open("extracted_text.json", "w", encoding="utf-8") as f:
    json.dump(page_data, f, ensure_ascii=False, indent=2)

print(f"✅ Extracted {len(page_data)} pages.")


✅ Extracted 22 pages.


In [ ]:
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load JSON dataset
with open("extracted_text.json", "r", encoding="utf-8") as f:
    pages = json.load(f)

# define chunks 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

docs = []
for p in pages:
    chunks = text_splitter.split_text(p["text"])
    for chunk in chunks:
        doc = Document(
            page_content=chunk,
            metadata={"page": p["page"]}
        )
        docs.append(doc)

print(f"✅ Total chunks: {len(docs)}")


✅ Total chunks: 82


In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# multilingual model for Thai
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# FAISS index: Facebook AI Similarity Search
db = FAISS.from_documents(docs, embedding_model)


db.save_local("faiss_afterburner_index")

print("✅ FAISS vectorstore saved.")


C:\Users\Asus\AppData\Local\Temp\ipykernel_48272\3419486862.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
d:\AirForce_project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ FAISS vectorstore saved.


In [6]:
# Load index
db = FAISS.load_local(
    "faiss_afterburner_index",
    embedding_model,
    allow_dangerous_deserialization=True
)


query = "การถอด metering valve ทำอย่างไร"
results = db.similarity_search(query, k=2)

for i, r in enumerate(results, 1):
    print(f"🔎 Chunk {i} (หน้า {r.metadata['page']}):") # which page of chunk
    print(r.page_content[:500])
    print("-" * 80)


🔎 Chunk 1 (หน้า 15):
Remove bypass valve (11) and insert (9) only if
screw (84) and preformed packings (86, 87). Break away
clogged or damaged. Remove standpipe (14)
torque shall be 2.0 pound-inch minimum when removing
and insert (15) only if damaged. Breakaway
linkage trim screw. Replace insert (15, Figure 3-15) if
torque shall be 4.0 pound-inch minimum when
breakaway torque is below limit.
removing bypass valve (11). Replace insert (9)
if breakaway torque is below limit.
3.20
REMOVAL OF NOZZLE CONTROL LIMIT
--------------------------------------------------------------------------------
🔎 Chunk 2 (หน้า 12):
pull out spacers (38) from bore of linkage
Exercise care not to overstretch the bellows
bracket.
while removing, as permanent deformation will
(11)
Carefully slide pin (40) out of the shaft of
occur.
sensor bellows (45); then remove screws (41).
(13)
Remove sensor bellows. After pin (Figure 3-9,
Removal torque for screws (41) shall be 1.5
40) has been removed, carefully remove bel

In [7]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA


llm = Ollama(model="mistral")


C:\Users\Asus\AppData\Local\Temp\ipykernel_48272\2897339608.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
db = FAISS.load_local("faiss_afterburner_index", embedding_model, allow_dangerous_deserialization=True)

# Retrieval chain (good for untouch data)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())


In [17]:
question = "ขั้นตอนถอด pilot burner regulator ต้องทำอย่างไร"

response = qa_chain.run(
    f"""คุณคือผู้ช่วยด้านเทคนิคภาษาไทย ช่วยตอบคำถามต่อไปนี้อย่างกระชับ:
\"{question}\""""
)

print(response)


เพื่อถอด Pilot Burner Regulator, คุณต้องทำตามขั้นตอน berikut:
1. ซ่อมให้สะอาดส่วนแยกกันโดยใช้สรีเปิด-cleaner P-D-680, Type II or III.
2. ถูกไปคืนและแดกทุกส่วนที่อยู่บนภาพ 3-1 ตามตัวอธิบาย:
   a. นำล็อคไวริงออกแล้วจัดทำใหม่โดยที่ไม่ใช่การขัดลอกไวริง
   b. ตัดส่วนล็อคไวริงออก
3. แก้ปุ่มหลัก (14, Figure 3-2) จากช่อที่บำกะเภา
   - ขณะนี้จะเห็นว่าส่วนลึกนี้มีส่วนที่แปลงชื่อ 14
4. ถูกไปคืนแล้วจัดทำภาพ 3-5
   a. นำรอเตอร์ (2, Figure 3-5) ออก
   b. หลีกบวมส่วนรอเตอร์ออกจากเซ็กชักนั้น, แล้วถูกไปคืนข้างนำรอเตอร์
5. ตัดส่วนภาพที่มีชื่อ 1 จากเซ็กชักนั้น, แล้วถูกไปคืนข้างนำรอเตอร์
6. นำเซ็กชักนั้น (1) ออกจากช่อที่บำกะเภา
7. ถูกไปคืนแล้วจัดทำภาพ 3-1
   a. นำซ้าย (2, Figure 3-1) ออกจากช่อที่บำกะเภา
   b. นำส่วนซ้าย (4) และผูกปัญหา (5), ปากที่แก้ปุ่มหลัก
8. ถูกไปคืนแล้วจัดทำภาพ 3-2
   a. ตัดส่วนช่อที่บำกะเภา (3) โดยใช้กันสะดวก Puller, PN 21C3605G001; แล้วเลิกซึ่ง
   b. ตัดส่วนช่อที่บำกะเภา (7)
9. ถูกไปคืนแล้วจัดทำภาพ 3-5
   a. นำส่วนซ้าย (2, Figure 3-5) ออกจากช่อที่บำกะเภา
   b. หลีกบวมส่วนรอเตอร์ออกจ